# Run prompt flow in Azure AI

**Requirements** - In order to benefit from this tutorial, you will need:
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed PromptFlow SDK

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your Azure AI workspace from the Python SDK
- Create and develop a new promptflow run
- Evaluate the run with a evaluation flow

**Motivations** - This guide will walk you through the main user journey of prompt flow code-first experience. You will learn how to create and develop your first prompt flow, test and evaluate it.

## 0. Install dependent packages

In [1]:
%pip install -r ../../requirements.txt

Looking in indexes: https://pypi.org/simple, https://azuremlsdktestpypi.azureedge.net/promptflow/
Note: you may need to restart the kernel to use updated packages.


# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [2]:
import json

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

# azure version promptflow apis
from promptflow.azure import PFClient

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [4]:
# Get a handle to workspace
pf = PFClient.from_config(credential=credential)

runtime = "demo-mir"  # TODO replace with your runtime name

Found the config file in: D:\enlistment\aml\promptflow\prompt-flow\examples\.azureml\config.json


## 1.4 Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `web-classification` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before.

Prepare your Azure Open AI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

Please go to [workspace portal](https://ml.azure.com/), click `Prompt flow` -> `Connections` -> `Create`, then follow the instruction to create your own connections. 
Learn more on [connections](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-connections?view=azureml-api-2).

## 2. Create a new run

`web-classification` is a flow demonstrating multi-class classification with LLM. Given an url, it will classify the url into one web category with just a few shots, simple summarization and classification prompts.

### Set flow path and input data

In [5]:
# load flow
flow = "../../flows/standard/web-classification"
data = "../../flows/standard/web-classification/data.jsonl"

### Submit run

In [6]:
# create run
base_run = pf.run(
    flow=flow,
    data=data,
    runtime=runtime,
)
print(base_run)

Portal url: https://ml.azure.com/prompts/flow/bulkrun/run/web_classification_default_20230818_174622_330767/details?wsid=/subscriptions/96aede12-2f73-41cb-b983-6d11a904839b/resourceGroups/promptflow/providers/Microsoft.MachineLearningServices/workspaces/promptflow-eastus&flight=promptfilestorage,PFSourceRun=false
name: web_classification_default_20230818_174622_330767
created_on: '2023-08-18T09:46:37.489886+00:00'
status: Running
display_name: web_classification_default_20230818_174622_330767
description: null
tags: {}
properties:
  azureml.promptflow.runtime_name: demo-mir
  azureml.promptflow.runtime_version: 20230808.v1
  azureml.promptflow.definition_file_name: flow.dag.yaml
  azureml.promptflow.snapshot_id: 773837e6-f837-4ca0-abce-8a1f7e143db2
creation_context:
  userObjectId: f7e94047-e084-4061-82ce-af31037a9d38
  userPuId: 10037FFE8983B2EB
  userIdp: null
  userAltSecId: null
  userIss: https://sts.windows.net/72f988bf-86f1-41af-91ab-2d7cd011db47/
  userTenantId: 72f988bf-86f1-4

In [7]:
pf.stream(base_run)

In [8]:
details = pf.get_details(base_run)
details.head(10)

InvalidRunStatusError: Run 'web_classification_default_20230818_174622_330767' is not completed, the status is 'Running'. Please wait for its completion, or select other completed run(s).

In [ ]:
pf.visualize(base_run)

## 3. Evaluate your flow run result
Then you can use an evaluation method to evaluate your flow. The evaluation methods are also flows which use Python or LLM etc., to calculate metrics like accuracy, relevance score.

In this notebook, we use `classification-accuracy-eval` flow to evaluate. This is a flow illustrating how to evaluate the performance of a classification system. It involves comparing each prediction to the groundtruth and assigns a "Correct" or "Incorrect" grade, and aggregating the results to produce metrics such as accuracy, which reflects how good the system is at classifying the data.

In [ ]:
eval_run = pf.run(
    flow="../../flows/evaluation/classification-accuracy-eval",
    data=data,
    run=base_run,
    column_mapping={
        "groundtruth": "${data.answer}",
        "prediction": "${run.outputs.category}",
    },
    runtime=runtime,
)

In [ ]:
pf.stream(eval_run)

In [ ]:
details = pf.get_details(eval_run)
details.head(10)

In [ ]:
metrics = pf.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([base_run, eval_run])

### Create another run with different variant node

In this example, `web-classification`'s node `summarize_text_content` has two variants: `variant_0` and `variant_1`. The difference between them is the inputs parameters:

variant_0:

    - inputs:
        - deployment_name: gpt-35-turbo
        - max_tokens: '128'
        - temperature: '0.2'
        - text: ${fetch_text_content_from_url.output}

variant_1:

    - inputs:
        - deployment_name: gpt-35-turbo
        - max_tokens: '256'
        - temperature: '0.3'
        - text: ${fetch_text_content_from_url.output}


You can check the whole flow definition at [flow.dag.yaml](../../flows/standard/web-classification/flow.dag.yaml)

In [ ]:
# use the variant1 of the summarize_text_content node.
variant_run = pf.run(
    flow=flow,
    data=data,
    variant="${summarize_text_content.variant_1}",  # here we specify node "summarize_text_content" to use variant 1 version.
    runtime=runtime,
)

In [ ]:
pf.stream(variant_run)

In [ ]:
details = pf.get_details(variant_run)
details.head(10)

### Run evaluation against variant run

In [ ]:
eval_flow = "../../flows/evaluation/classification-accuracy-eval"

eval_run_variant = pf.run(
    flow=eval_flow,
    data="../../flows/standard/web-classification/data.jsonl",  # path to the data file
    run=variant_run,  # use run as the variant
    column_mapping={
        # reference data
        "groundtruth": "${data.answer}",
        # reference the run's output
        "prediction": "${run.outputs.category}",
    },
    runtime=runtime,
)

In [ ]:
pf.stream(eval_run_variant)

In [ ]:
details = pf.get_details(eval_run_variant)
details.head(10)

In [ ]:
metrics = pf.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

In [ ]:
pf.visualize([eval_run, eval_run_variant])

# Next Steps

Learn more on how to:
- run the flow as a component in a azureml pipeline: [flow in pipeline](../flow-in-pipeline/pipeline.ipynb).